In [1]:
### (참고링크) <https://wikidocs.net/157586>
### Lightning Module은 6가지로 구성됩니다.
 # ** Computations (init)
 # ** Train loop (training_step)
 # ** Validation loop (validation_step)
 # ** Test loop (test_step)
 # ** Prediction loop (predict_step)
 # ** Optimizers (configure_optimizers)

In [3]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 705 kB 35.2 MB/s 
     |████████████████████████████████| 5.9 MB 58.5 MB/s 
     |████████████████████████████████| 419 kB 63.8 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is incompatible.


In [4]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from sklearn.datasets import load_boston
from sklearn.preprocessing import MinMaxScaler

In [6]:
sk_X, sk_y = load_boston(return_X_y=True)
print(X[0])

[6.320e-03 1.800e+01 2.310e+00 0.000e+00 5.380e-01 6.575e+00 6.520e+01
 4.090e+00 1.000e+00 2.960e+02 1.530e+01 3.969e+02 4.980e+00]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

In [7]:
class SklearnDataset(Dataset) :
    def __init__(self, X: np.ndarray, y) :
        super().__init__()
        scaler = MinMaxScaler()

        scaler.fit(X)
        self.X = scaler.transform(X)
        self.y = y

    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx) :
        X = self.X[idx].astype(np.float32)
        y = self.y[idx].astype(np.float32)
        return X, y

In [8]:
bostons = SklearnDataset(sk_X, sk_y)
train_loader = DataLoader(bostons, batch_size=32, shuffle=True, drop_last=True)

In [12]:
class LinRegModel(pl.LightningModule):
    def __init__(self, input_dim: int) :
        super().__init__()
        self.linear = torch.nn.Linear(
            in_features = 13, out_features=1, bias=True
        )
    def forward(self, X) :
        y_hat = self.linear(X)
        return y_hat
    def training_step(self, batch, batch_idx) :
        X, y = batch
        X = X.view(X.size(0), -1)
        y_hat = self(X)
        loss = torch.nn.functional.mse_loss(y_hat, y, reduction='sum')
        return loss
    def configure_optimizers(self) :
        return torch.optim.Adam(self.parameters(), lr=1e-4)


In [13]:
trainer = pl.Trainer()
model = LinRegModel(input_dim=13)
trainer.fit(model, train_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1769: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  category=PossibleUserWarning,
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:92: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  category=PossibleUserWarning,
INFO:pytorch_lightning.callbacks.model_summary:
  | Name   | Type   | Params
----------------------------------
0 | linear | Linear | 14    
----------------------------------
14        Trainab

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1000` reached.
